In [ ]:
{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 1,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "2.4654328284286695\n"
     ]
    }
   ],
   "source": [
    "from numpy.core.fromnumeric import var\n",
    "import pandas as pd\n",
    "import numpy as np\n",
    "from pandas.core.algorithms import rank\n",
    "from xgboost import XGBRegressor\n",
    "from sklearn.ensemble import RandomForestRegressor\n",
    "from sklearn.metrics import mean_absolute_error\n",
    "from sklearn.model_selection import train_test_split\n",
    "from sklearn.preprocessing import OneHotEncoder\n",
    "from sklearn.impute import SimpleImputer\n",
    "from sklearn.tree import DecisionTreeRegressor\n",
    "\n",
    "dataPath = 'csv/wineDataNew.csv' # This dataset has reduced country and variety cardinality\n",
    "wineData = pd.read_csv(dataPath)\n",
    "\n",
    "y = wineData.points\n",
    "features = ['country','variety']\n",
    "X = wineData[features]\n",
    "\n",
    "# Split Into Validation and Training Data\n",
    "X_train, X_valid, Y_train, Y_valid = train_test_split(X, y, random_state=1)\n",
    " \n",
    "# Apply One-Hot Encoder\n",
    "OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)\n",
    "OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[features]))\n",
    "OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[features]))\n",
    " \n",
    "# One-hot encoding Removed Index...Put It Back\n",
    "OH_cols_train.index = X_train.index\n",
    "OH_cols_valid.index = X_valid.index\n",
    "\n",
    "\n",
    "# ***************************************************************************************************************************\n",
    "#                                                 Random Forest Model                                                       #\n",
    "# ***************************************************************************************************************************\n",
    "rf_model = RandomForestRegressor(random_state=0) \n",
    "rf_model.fit(OH_cols_train, Y_train)\n",
    "rf_val_predictions = rf_model.predict(OH_cols_valid)\n",
    "rf_val_mae = mean_absolute_error(rf_val_predictions, Y_valid)\n",
    " \n",
    "# print(rf_val_mae) # MAE 2.364 with original country cardinality\n",
    "# print(rf_val_mae) # MAE 2.370 with reduced country cardinality\n",
    "# print(rf_val_mae) # MAE 2.465 with reduced country and variety cardinality\n",
    "\n",
    "\n",
    "# ***************************************************************************************************************************\n",
    "#                                        XGB Regressor Model w/ MAE Averages                                                #\n",
    "# ***************************************************************************************************************************\n",
    "xgb_model = XGBRegressor()\n",
    "xgb_model = XGBRegressor(n_estimators=500,learning_rate=0.05,n_jobs=4)\n",
    "xgb_model.fit(OH_cols_train, Y_train, \n",
    "              early_stopping_rounds=5,\n",
    "              eval_set=[(OH_cols_valid, Y_valid)],\n",
    "              verbose=False)\n",
    "predictions = xgb_model.predict(OH_cols_valid)\n",
    "# xgb_mae = mean_absolute_error(predictions, Y_valid)\n",
    "# print(xgb_mae) # MAE 2.39 \n",
    "# Reducing the country/variety cardinality from 10/632 to 10/10 has not significantly altered the MAE, but has greatly reduced the runtime\n",
    "# print(xgb_mae) # MAE 2.466 - 11s runtime\n",
    "\n"
   ]
  }
 ],
 "metadata": {
  "interpreter": {
   "hash": "4477214b5051ca77ffcee0215f1b66b78859f8d228c301db64d6a137f8649988"
  },
  "kernelspec": {
   "display_name": "Python 3.9.9 64-bit (windows store)",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.9.9"
  },
  "orig_nbformat": 4
 },
 "nbformat": 4,
 "nbformat_minor": 2
}